# 分析只用含氧血

In [ ]:
# import 必要東西
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import glob

### 1. 定義minmax函數，以做正規化

In [ ]:
def minmax(df_temp):
    dfs =  (df_temp - df_temp.min())/(df_temp.max() - df_temp.min())
    return dfs

### 2. 定義資料處理函數以及合併函數 - **No Minmax**
   *  Region 1 => 25-28, 36-38, 46-49
   *  Region 2 => 22-24, 32-35, 43-45, 29-31, 39-42, 50-52

In [ ]:
def get_data(path):

    df = pd.read_csv(path, skiprows= 40)

    df_drop = df.drop(columns= ["BodyMovement", "RemovalMark", 'PreScan', 'Mark', 'Probe1(Oxy)'])
    df_drop = df_drop.set_index("Time")
    index_list = [int(x.split(':')[1])*60 + float(x.split(':')[2]) if not '-23' in x else -999 for x in df_drop.index]
    df_drop.index = index_list
    try:
        df_drop = df_drop.drop([-999],axis=0)
    except:
        pass

    # 10筆1秒
    df_new = df_drop


    region_1 = df_new[['CH25', 'CH26', 'CH27', 'CH28', 'CH36', 'CH37', 'CH38', 'CH46' , 'CH47', 'CH48', 'CH49']].mean(axis=1)
    region_2 = df_new[['CH22', 'CH23', 'CH24', 'CH32', 'CH33', 'CH34', 'CH35', 'CH43', 'CH44', 'CH45', 'CH29', 'CH30', 'CH31', 'CH39', 'CH40', 'CH41', 'CH42', 'CH50', 'CH51', 'CH52']].mean(axis=1)
    dff = pd.concat([region_1, region_2], axis=1)

    return dff

### 3. 定義資料處理函數以及合併函數 - **Minmax**

In [ ]:
def get_data_minmax(path):

    df = pd.read_csv(path, skiprows= 40)

    df_drop = df.drop(columns= ["BodyMovement", "RemovalMark", 'PreScan', 'Mark', 'Probe1(Oxy)'])
    df_drop = df_drop.set_index("Time")
    index_list = [int(x.split(':')[1])*60 + float(x.split(':')[2]) if not '-23' in x else -999 for x in df_drop.index]
    df_drop.index = index_list
    try:
        df_drop = df_drop.drop([-999],axis=0)
    except:
        pass

    # 10筆1秒
    df_new = df_drop


    region_1 = df_new[['CH25', 'CH26', 'CH27', 'CH28', 'CH36', 'CH37', 'CH38', 'CH46' , 'CH47', 'CH48', 'CH49']].mean(axis=1)
    region_2 = df_new[['CH22', 'CH23', 'CH24', 'CH32', 'CH33', 'CH34', 'CH35', 'CH43', 'CH44', 'CH45', 'CH29', 'CH30', 'CH31', 'CH39', 'CH40', 'CH41', 'CH42', 'CH50', 'CH51', 'CH52']].mean(axis=1)
    dff = pd.concat([region_1, region_2], axis=1)

    return minmax(dff)


### 4. 叫出函數(兩種族群 * 兩種處理方式 = 4次)

In [ ]:
health_list = glob.glob(r".\正常人_LFT_data_N_33\*_Oxy.csv")

for csv_path in health_list:

    picture_filename = csv_path.split("\\")[-1].split("_")[0]
    df_temp = get_data_minmax(csv_path)
    # print(len(df_temp))

    if len(df_temp) < 1250:
        print(picture_filename)
        continue

    np.save(f'nominmax_health/{picture_filename}_HBA_Probe1_Oxy_region_nominmax', df_temp)


In [ ]:
health_list = glob.glob(r".\正常人_LFT_data_N_33\*_Oxy.csv")

for csv_path in health_list:

    picture_filename = csv_path.split("\\")[-1].split("_")[0]
    df_temp = get_data(csv_path)
    # print(len(df_temp))

    if len(df_temp) < 1250:
        print(picture_filename)
        continue

    np.save(f'region_health/{picture_filename}_HBA_Probe1_Oxy_region_nominmax', df_temp)

In [ ]:
un_health_list = glob.glob(r".\病人組_LFT_data_N_32\*_Oxy.csv")

for csv_path in un_health_list:

    picture_filename = csv_path.split("\\")[-1].split("_")[0]
    df_temp = get_data(csv_path)
    # print(len(df_temp))

    np.save(f'region_unhealth/{picture_filename}_HBA_Probe1_Oxy_region_nominmax', df_temp)

In [ ]:
un_health_list = glob.glob(r".\病人組_LFT_data_N_32\*_Oxy.csv")

for csv_path in un_health_list:

    picture_filename = csv_path.split("\\")[-1].split("_")[0]
    df_temp = get_data_minmax(csv_path)
    # print(len(df_temp))

    np.save(f'nominmax_unhealth/{picture_filename}_HBA_Probe1_Oxy_region_nominmax', df_temp)